In [1]:
import os

In [2]:
%pwd

'/Users/shreyasshah/Desktop/kidney-disease-classification-deep-learning/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/shreyasshah/Desktop/kidney-disease-classification-deep-learning'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
  root_dir: Path
  source_URL: str
  local_data_file: Path
  unzip_dir: Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
  def __init__(self, config_filepath=CONFIG_FILE_PATH,
               params_filepath=PARAMS_FILE_PATH) -> None:
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    create_directories([self.config.artifacts_root])

  def get_data_ingestion_config(self) -> DataIngestionConfig:
    config = self.config.data_ingestion
    create_directories([config.root_dir])
    data_ingestion_config = DataIngestionConfig(
        root_dir=config.root_dir, source_URL=config.source_URL, local_data_file=config.local_data_file, unzip_dir=config.unzip_dir)
    return data_ingestion_config

In [9]:
import os, zipfile, gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

/Users/shreyasshah/Desktop/kidney-disease-classification-deep-learning/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [10]:
class DataIngestion:
  def __init__(self, config: DataIngestionConfig):
    self.config = config

  def download_file(self) -> str:
    '''
    Fetch data from the url
    '''

    try:
      dataset_url = self.config.source_URL
      zip_download_dir = self.config.local_data_file
      os.makedirs("artifacts/data_ingestion", exist_ok=True)
      logger.info(
          f"Downloading data from {dataset_url} into file {zip_download_dir}")

      file_id = dataset_url.split("/")[-2]
      prefix = 'https://drive.google.com/uc?/export=download&id='
      gdown.download(prefix+file_id, zip_download_dir)

      logger.info(
          f"Downloaded data from {dataset_url} into file {zip_download_dir}")

    except Exception as e:
      raise e

  def extract_zip_file(self):
    """
    zip_file_path: str
    Extracts the zip file into the data directory
    Function returns None
    """
    unzip_path = self.config.unzip_dir
    os.makedirs(unzip_path, exist_ok=True)
    with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
      zip_ref.extractall(unzip_path)

In [11]:
try:
  config = ConfigurationManager()
  data_ingestion_config = config.get_data_ingestion_config()
  data_ingestion = DataIngestion(config=data_ingestion_config)
  data_ingestion.download_file()
  data_ingestion.extract_zip_file()
except Exception as e:
  raise e

[2024-04-18 18:39:08,986: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-18 18:39:08,988: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-18 18:39:08,989: INFO: common: created directory at artifacts]
[2024-04-18 18:39:08,991: INFO: common: created directory at artifacts/data_ingestion]
[2024-04-18 18:39:08,992: INFO: 2646887694: Downloading data from https://drive.google.com/file/d/1qlKHoCcecTkCSpDuKD24NiSr9PJhjXSh/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1qlKHoCcecTkCSpDuKD24NiSr9PJhjXSh
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1qlKHoCcecTkCSpDuKD24NiSr9PJhjXSh&confirm=t&uuid=53ce90ef-7766-429b-b2ae-c493044287a7
To: /Users/shreyasshah/Desktop/kidney-disease-classification-deep-learning/artifacts/data_ingestion/data.zip
100%|██████████| 57.7M/57.7M [00:44<00:00, 1.28MB/s]

[2024-04-18 18:40:10,286: INFO: 2646887694: Downloaded data from https://drive.google.com/file/d/1qlKHoCcecTkCSpDuKD24NiSr9PJhjXSh/view?usp=sharing into file artifacts/data_ingestion/data.zip]
